# transformers fine tune for model: bert-base-cased dataset: yelp_review_full


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import evaluate
import inspect
from datetime import datetime

# helper debug method
def print_arg(value, msg=None):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_prefix = f"[BYDEBUG:{timestamp}]:"

    # Get the caller's frame
    frame = inspect.currentframe().f_back
    try:
        # Find the argument name in the caller's frame
        for k, v in frame.f_locals.items():
            if v is value:
                print(f"{log_prefix}{msg}:{k} = {value!r}")
                return
        # If not found in locals, check globals
        for k, v in frame.f_globals.items():
            if v is value:
                print(f"{log_prefix}{msg}:{k} = {value!r}")
                return
        print(f"{log_prefix}{msg}:<unknown> = {value!r}")
    finally:
        del frame  # Avoid reference cycles


# load dataset
dataset = load_dataset("yelp_review_full")
print_arg(dataset, "loaded dataset")
print_arg(dataset["train"][100], "loaded dataset train sample")
print_arg(dataset["test"][100], "loaded dataset test sample")

# prepare dataset for training
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print_arg(tokenizer)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print_arg(tokenized_datasets, "tokenized datasets")

train_data_size = 10
eval_data_size = 10
test_data_size = 5
print_arg(train_data_size, "train_data_size")
print_arg(eval_data_size, "eval_data_size")
print_arg(test_data_size, "test_data_size")

# pick some small subset for quick training/evaluation
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(train_data_size))
print_arg(small_train_dataset)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(eval_data_size))
print_arg(small_eval_dataset)

# load the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
print_arg(model, "loaded model")

# define training arguments
model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps is set to 100, which means the model will log training information every 100 steps
# eval_strategy is set to "epoch", meaning evaluation will be done at the end of each epoch
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30,
                                  save_total_limit=5,
                                  eval_strategy="epoch")

print_arg(training_args, "training arguments")

# define evaluation metric
metric = evaluate.load("accuracy")
print_arg(metric, "evaluation metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
print_arg(trainer, "starting training...")

# monitor GPU usage
# watch -n 1 nvidia-smi

# start training
print_arg(trainer.state, "trainer state before training")
trainer.train()
print_arg(trainer.state, "trainer state after training")

# evaluate the model
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(test_data_size))
print_arg(small_test_dataset)

trainer.evaluate(small_test_dataset)
print_arg(trainer.state, "trainer state after evaluation")

# save the model
trainer.save_model(model_dir)
print_arg(model_dir, "saved model")

# save the training state
trainer.save_state()
print_arg(model_dir, "saved training state")

# save the tokenizer
# tokenizer.save_pretrained(model_dir)


[BYDEBUG:2025-07-28 22:19:12]:loaded dataset:dataset = DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})
[BYDEBUG:2025-07-28 22:19:12]:loaded dataset train sample:<unknown> = {'label': 0, 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager w

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[BYDEBUG:2025-07-28 22:19:15]:loaded model:model = BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


Using the latest cached version of the module from C:\Users\bill\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Mon Jul 28 19:51:14 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


[BYDEBUG:2025-07-28 22:19:20]:evaluation metric:metric = EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value('int32'), 'references': Value('int32')}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
    

...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.592852,0.400000
2,No log,1.574741,0.400000
3,No log,1.555039,0.400000


C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\dev\inst\miniconda3\env

[BYDEBUG:2025-07-28 22:22:02]:trainer state after training:<unknown> = TrainerState(epoch=3.0, global_step=3, max_steps=3, logging_steps=30, eval_steps=500, save_steps=500, train_batch_size=16, num_train_epochs=3, num_input_tokens_seen=0, total_flos=7893544273920.0, log_history=[{'eval_loss': 1.5928515195846558, 'eval_accuracy': 0.4, 'eval_runtime': 7.9897, 'eval_samples_per_second': 1.252, 'eval_steps_per_second': 0.25, 'epoch': 1.0, 'step': 1}, {'eval_loss': 1.5747413635253906, 'eval_accuracy': 0.4, 'eval_runtime': 9.0192, 'eval_samples_per_second': 1.109, 'eval_steps_per_second': 0.222, 'epoch': 2.0, 'step': 2}, {'eval_loss': 1.5550391674041748, 'eval_accuracy': 0.4, 'eval_runtime': 9.2172, 'eval_samples_per_second': 1.085, 'eval_steps_per_second': 0.217, 'epoch': 3.0, 'step': 3}, {'train_runtime': 161.2564, 'train_samples_per_second': 0.186, 'train_steps_per_second': 0.019, 'total_flos': 7893544273920.0, 'train_loss': 1.5614074071248372, 'epoch': 3.0, 'step': 3}], best_metric=None,

C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[BYDEBUG:2025-07-28 22:22:07]:trainer state after evaluation:<unknown> = TrainerState(epoch=3.0, global_step=3, max_steps=3, logging_steps=30, eval_steps=500, save_steps=500, train_batch_size=16, num_train_epochs=3, num_input_tokens_seen=0, total_flos=7893544273920.0, log_history=[{'eval_loss': 1.5928515195846558, 'eval_accuracy': 0.4, 'eval_runtime': 7.9897, 'eval_samples_per_second': 1.252, 'eval_steps_per_second': 0.25, 'epoch': 1.0, 'step': 1}, {'eval_loss': 1.5747413635253906, 'eval_accuracy': 0.4, 'eval_runtime': 9.0192, 'eval_samples_per_second': 1.109, 'eval_steps_per_second': 0.222, 'epoch': 2.0, 'step': 2}, {'eval_loss': 1.5550391674041748, 'eval_accuracy': 0.4, 'eval_runtime': 9.2172, 'eval_samples_per_second': 1.085, 'eval_steps_per_second': 0.217, 'epoch': 3.0, 'step': 3}, {'train_runtime': 161.2564, 'train_samples_per_second': 0.186, 'train_steps_per_second': 0.019, 'total_flos': 7893544273920.0, 'train_loss': 1.5614074071248372, 'epoch': 3.0, 'step': 3}, {'eval_loss': 1.